In [9]:
import sys

sys.path.append("../..")
from cge_modeling import Variable, Parameter, Equation, CGEModel
import pymc as pm
import numpy as np
from cge_modeling.tools.sympy_tools import expand_obj_by_indices

# Case 1: Square Matrix

## Vectorized solution

In [2]:
X = pm.draw(pm.NegativeBinomial.dist(mu=100, alpha=10, shape=(3, 3)))
P_X = 1 + pm.draw(pm.Beta.dist(alpha=1, beta=3, size=(3,)))

In [3]:
VC = X.sum(axis=0)
phi_X = X / VC
P_VC = (X * P_X[:, None]).sum(axis=0) / VC

In [4]:
np.allclose(VC * P_VC, (X * P_X[:, None]).sum(axis=0))

True

So I just want to sum along axis 0 (the rows), paying attention to make sure the prices broadcast correctly. 

If I don't reshape the prices of X I get the wrong answer:

In [5]:
np.allclose(VC * P_VC, (X * P_X).sum(axis=0))

False

## Sympy summation

### First try: Naive

Type in the equation directly. Remember the desired output is:

$$VC_j P_{VC, j} = X_{1, j} P_1 + X_{2,j} P_2 + X_{3,j}$$

That is: 
   - The LHS index matches the 2nd index on the RHS
   - All 3 prices show up
   - The price index matches the first index of each X.

In [28]:
coords = {"i": ["A", "B", "C"], "j": ["A", "B", "C"]}

x = Variable("X", dims=["i", "j"], description="Demand for <dim:i> goods by <dim:j>")
p_x = Variable("P_X", dims=["i"], description="Price of intermediate good <dim:i>")
vc = Variable("VC", dims=["i"], description="Value chain of the <dim:i> sector")
p_vc = Variable("P_VC", dims=["i"], description="Price of <dim:i> sector value chain")

psi_x = Parameter("psi_X", dims=["i", "j"])

eq = Equation("Value chain of section <dim:i>", "VC * P_VC = Sum(P_X * X, (j, 0, 2))")

In [29]:
mod = CGEModel(compile=False, coords=coords)
mod.add_variables([x, p_x, vc, p_vc])
mod.add_parameters([psi_x])
mod.add_sympy_equation(eq)

Here the RHS indexes are backwards, and the prices are not correct.

In [30]:
for eq in expand_obj_by_indices(mod.equations[0], coords, dims=None):
    display(eq.symbolic_eq.doit())

Eq(P_VC[A]*VC[A], (X[A, 0] + X[A, 1] + X[A, 2])*P_X[A])

Eq(P_VC[B]*VC[B], (X[B, 0] + X[B, 1] + X[B, 2])*P_X[B])

Eq(P_VC[C]*VC[C], (X[C, 0] + X[C, 1] + X[C, 2])*P_X[C])

### Second try: limited transpose

Flip i and j on P_X

In [40]:
eq = Equation("Value chain of section <dim:i>", "VC * P_VC = Sum(P_X.subs({i:j}) * X, (j, 0, 2))")

mod = CGEModel(compile=False, coords=coords)
mod.add_variables([x, p_x, vc, p_vc])
mod.add_parameters([psi_x])
mod.add_sympy_equation(eq)

This is actually quite close, but we're summing down the columns not the rows. Notice that the first index of X is fixed, so we're "sliding" along the 2nd index, which is the columns. This is equivalent to `sum(axis=1)`.

In [42]:
mod.equations[0]

Value chain of section <dim:i>

In [43]:
for eq in expand_obj_by_indices(mod.equations[0], coords, dims=None):
    display(eq.symbolic_eq.doit())

Eq(P_VC[A]*VC[A], P_X[0]*X[A, 0] + P_X[1]*X[A, 1] + P_X[2]*X[A, 2])

Eq(P_VC[B]*VC[B], P_X[0]*X[B, 0] + P_X[1]*X[B, 1] + P_X[2]*X[B, 2])

Eq(P_VC[C]*VC[C], P_X[0]*X[C, 0] + P_X[1]*X[C, 1] + P_X[2]*X[C, 2])

### Third try: Transpose everything

So this is the only way I know it works. Pretty ugly though.

In [47]:
eq = Equation(
    "Value chain of section <dim:i>",
    "(VC * P_VC) = Sum(P_X.subs({i:j}) * X.subs([(i,z), (j, i), (z,j)]), (j, 0, 2))",
)
mod = CGEModel(compile=False, coords=coords)
mod.add_variables([x, p_x, vc, p_vc])
mod.add_parameters([psi_x])
mod.add_sympy_equation(eq)

In [48]:
mod.equations[0]

Value chain of section <dim:i>

In [49]:
for eq in expand_obj_by_indices(mod.equations[0], coords, dims=None):
    display(eq.symbolic_eq.doit())

Eq(P_VC[A]*VC[A], P_X[0]*X[0, A] + P_X[1]*X[1, A] + P_X[2]*X[2, A])

Eq(P_VC[B]*VC[B], P_X[0]*X[0, B] + P_X[1]*X[1, B] + P_X[2]*X[2, B])

Eq(P_VC[C]*VC[C], P_X[0]*X[0, C] + P_X[1]*X[1, C] + P_X[2]*X[2, C])

In [36]:
eq = Equation(
    "Value chain of section <dim:i>",
    "VC * P_VC = Sum(P_X.subs({i:j}) * X.subs([(i,z), (j, i), (z,j)]), (j, 0, 2))",
)
mod = CGEModel(compile=False, coords=coords)
mod.add_variables([x, p_x, vc, p_vc])
mod.add_parameters([psi_x])
mod.add_sympy_equation(eq)

In [37]:
for eq in expand_obj_by_indices(mod.equations[0], coords, dims=None):
    display(eq.symbolic_eq.doit())

Eq(P_VC[A]*VC[A], P_X[0]*X[0, A] + P_X[1]*X[1, A] + P_X[2]*X[2, A])

Eq(P_VC[B]*VC[B], P_X[0]*X[0, B] + P_X[1]*X[1, B] + P_X[2]*X[2, B])

Eq(P_VC[C]*VC[C], P_X[0]*X[0, C] + P_X[1]*X[1, C] + P_X[2]*X[2, C])

# Non-square case

Now suppose we have a situation where the market isn't symmetric -- the `j` firms buy from the `i` firms, but NOT vice-versa.

In [50]:
X = pm.draw(pm.NegativeBinomial.dist(mu=100, alpha=10, shape=(3, 2)))
P_X = 1 + pm.draw(pm.Beta.dist(alpha=1, beta=3, size=(3,)))

VC = X.sum(axis=0)
phi_X = X / VC
P_VC = (X * P_X[:, None]).sum(axis=0) / VC

In the vectorized case, nothing changes.

In [51]:
np.allclose(VC * P_VC, (X * P_X[:, None]).sum(axis=0))

True

In the Sympy case, we now get shape errors if we try to transpose I think

In [59]:
coords = {"i": ["A", "B", "C"], "j": ["D", "E"]}

vc = Variable("VC", dims=["j"], description="Value chain of the <dim:j> sector")
p_vc = Variable("P_VC", dims=["j"], description="Price of <dim:j> sector value chain")

eq = Equation(
    "Value chain of section <dim:i>",
    "(VC * P_VC).subs({i:j}) = Sum(P_X * X.subs([(i,z), (j, i), (z,j)]), (i, 0, 2))",
)
mod = CGEModel(compile=False, coords=coords)
mod.add_variables([x, p_x, vc, p_vc])
mod.add_parameters([psi_x])
mod.add_sympy_equation(eq)

In [61]:
mod.equations[0]

Value chain of section <dim:i>

In [60]:
for eq in expand_obj_by_indices(mod.equations[0], coords, dims=None):
    display(eq.symbolic_eq.doit())

Eq(P_VC[D]*VC[D], P_X[0]*X[D, 0] + P_X[1]*X[D, 1] + P_X[2]*X[D, 2])

Eq(P_VC[E]*VC[E], P_X[0]*X[E, 0] + P_X[1]*X[E, 1] + P_X[2]*X[E, 2])

In [62]:
eq = Equation("Value chain of section <dim:i>", "VC * P_VC = Sum(P_X * X, (i, 0, 2))")
mod = CGEModel(compile=False, coords=coords)
mod.add_variables([x, p_x, vc, p_vc])
mod.add_parameters([psi_x])
mod.add_sympy_equation(eq)

In [63]:
for eq in expand_obj_by_indices(mod.equations[0], coords, dims=None):
    display(eq.symbolic_eq.doit())

Eq(P_VC[D]*VC[D], P_X[0]*X[0, D] + P_X[1]*X[1, D] + P_X[2]*X[2, D])

Eq(P_VC[E]*VC[E], P_X[0]*X[0, E] + P_X[1]*X[1, E] + P_X[2]*X[2, E])